## Create dataframe from sign_props.json file 

In [2]:
import json
import pandas as pd

data_dict = {}
columns = []
with open('new-data/sign_props.json') as json_file:
    data = json.load(json_file)    
    #print(data)
    row_index = 1
    for attr in data[0]:
        columns.append(attr)
    for record in data:
        row = []
        for attr in record:
            row.append(record[attr])
        data_dict[row_index] = row
        row_index += 1
        
#print(data_dict)
df = pd.DataFrame.from_dict(data_dict, orient='index', columns=columns)
df

,index,EntryID,LemmaID,Code,SignFrequency(M),SignFrequency(SD),SignFrequency(Z),SignFrequency(N),PercentUnknown,SignFrequency(M-Native),...,Transparency Z,H index,SignBankID,Parameter.Neighborhood.Density.2.0,Neighborhood Density 2.0,Parameter.Neighborhood.Density.2.0.1,Neighborhood Density 2.0.1,DominantTranslation,SignBankID.1,Neighborhood Density
1,0,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,None,NaN,NaN,NaN,NaN,None,None,0
2,1,&=arms-cross,&=arms-cross,J_01_098,3.467,2.177,-0.458,30.0,0.0,3.400,...,NaN,NaN,&=arms-cross\t,0.0,3.0,0.0,3.0,None,&=arms-cross\t,0
3,2,&=raise-hand,&=raise-hand,H_02_048,5.154,1.891,0.544,28.0,0.0,5.500,...,NaN,NaN,&=raise-hand\t,0.0,3.0,0.0,3.0,None,&=raise-hand\t,0
4,3,&=sigh,&=sigh,H_03_086,3.931,1.751,-0.196,30.0,0.0,3.267,...,NaN,NaN,&=sigh\t,0.0,3.0,0.0,3.0,None,&=sigh\t,0
5,4,1_dollar,1_dollar,C_02_025,3.034,1.500,-0.775,29.0,0.0,2.944,...,NaN,NaN,DOLLAR,9.0,0.0,9.0,0.0,None,DOLLAR,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2725,2724,zero,zero,D_03_036,4.192,1.415,0.051,26.0,0.0,4.313,...,NaN,NaN,ZERO,19.0,5.0,19.0,5.0,None,ZERO,6
2726,2725,zipper,zipper,D_01_034,3.148,1.610,-0.556,27.0,0.0,2.875,...,NaN,NaN,ZIPPER,0.0,0.0,0.0,0.0,None,ZIPPER,0
2727,2726,zoo,zoo,E_03_042,3.533,1.776,-0.346,30.0,0.0,3.857,...,NaN,NaN,None,4.0,3.0,4.0,3.0,None,None,3
2728,2727,zoom_in,zoom_in,B_02_056,3.958,1.781,-0.443,24.0,0.0,3.929,...,1.0,-0.19,ZOOM-IN\t,38.0,0.0,38.0,0.0,zoom_in,ZOOM-IN\t,0


## Finding categorical and range attributes

In [3]:
numerical_attr = list(df.select_dtypes(include=['float', 'int']).columns)
categorical_attr = []
for column in list(df.columns):
    if column not in numerical_attr:
        categorical_attr.append(column)   
print(numerical_attr)
print(categorical_attr)

['index', 'SignFrequency(M)', 'SignFrequency(SD)', 'SignFrequency(Z)', 'SignFrequency(N)', 'PercentUnknown', 'SignFrequency(M-Native)', 'SignFrequency(SD-Native)', 'SignFreq(Z-native)', 'SignFrequency(N-Native)', 'PercentUnknown(Native)', 'PercentGlossAgreement', 'PercentGlossAgreement(Native)', 'Iconicity(M)', 'Iconicity(SD)', 'Iconicity(Z)', 'Iconicity(N)', 'SubtLexUSLog10WF', 'SignOnset(ms)', 'SignOffset(ms)', 'SignLength(ms)', 'ClipLength(ms)', 'D.Iconicity(M) all', 'D.Iconicity(SD) all', 'D.Iconicity(N) all', 'D.Iconicity(Z) all', 'D.Iconicity(M) native', 'D.Iconicity(SD) native', 'D.Iconicity(N) native', 'D.Iconicity(Z) native', 'Initialized.2.0', 'FingerspelledLoanSign.2.0', 'Compound.2.0', 'NumberOfMorphemes.2.0', 'MarkedHandshape.2.0', 'FlexionChange.2.0', 'Spread.2.0', 'SpreadChange.2.0', 'ThumbContact.2.0', 'Contact.2.0', 'UlnarRotation.2.0', 'bglm_aoa', 'empirical_aoa', 'SignType.2.0Frequency', 'MajorLocation.2.0Frequency', 'MinorLocation.2.0Frequency', 'SecondMinorLocation

## Find the constraints of the attributes 
### For categorical attributes we find all the possible values and for numberical attributes we find min and max 

In [4]:
import math 
import pprint

constraints = {}
for attr in categorical_attr:
    if attr not in ['EntryID', 'LemmaID', 'Code', 'Iconicity_ID']:
        columnsData = df.loc[ : , attr ]
        constraints[attr] = list(set(list(columnsData)))
        
for attr in numerical_attr:    
    #drop nan values 
    columnsData = df.loc[ : , attr ].dropna()
    constraints[attr] = {}  
    #print(math.floor(min(list(columnsData))), math.ceil(max(list(columnsData))), attr)
    constraints[attr]['min'] = math.floor(min(list(columnsData)))
    constraints[attr]['max'] = math.ceil(max(list(columnsData)))

pprint.pprint(constraints)    

{'ClipLength(ms)': {'max': 3737, 'min': 701},
 'Compound.2.0': {'max': 1, 'min': 0},
 'Contact.2.0': {'max': 1, 'min': 0},
 'Contact.2.0Frequency': {'max': 1694, 'min': 1030},
 'D.Iconicity(M) all': {'max': 7, 'min': 1},
 'D.Iconicity(M) native': {'max': 7, 'min': 1},
 'D.Iconicity(N) all': {'max': 31, 'min': 26},
 'D.Iconicity(N) native': {'max': 22, 'min': 18},
 'D.Iconicity(SD) all': {'max': 3, 'min': 0},
 'D.Iconicity(SD) native': {'max': 3, 'min': 0},
 'D.Iconicity(Z) all': {'max': 3, 'min': -2},
 'D.Iconicity(Z) native': {'max': 3, 'min': -2},
 'DominantTranslation': ['pass_out',
                         'dryer',
                         'stumble',
                         'credit_card_charge',
                         'hurdle_skip',
                         'mechanic_2',
                         'same_time',
                         'dump',
                         'pile',
                         'theater',
                         'soon',
                         'go_through_h

                  'BAG',
                  'PRINT',
                  'TEACHER',
                  'COLOR',
                  'ICESKATE',
                  'WHIP',
                  'SOCIALIZE',
                  'EUROPE',
                  'BLOW',
                  'TALL-PERSON',
                  'FOLD',
                  'MADstr',
                  "DON'T-MIND\t",
                  'RUSSIA',
                  'BELTBAND',
                  'KISSck',
                  'ISLAND',
                  'LOOK-LIKE',
                  'BOOTbb',
                  'BRACELET',
                  'BULL',
                  'DENT',
                  'IMAGINE',
                  'CHEAP',
                  'LOUSY',
                  'WHY',
                  'COMPLAIN',
                  'PET',
                  '~HURRICANE\t',
                  'LEND',
                  'BELIEVEix',
                  'CONCEPT',
                  'INTERVENE',
                  'SIT',
                  'SCISSORS-CUT',
  

## Store attribute constraints in a json file

In [6]:
import json

with open('constraints.json', 'w') as fp:
    json.dump(constraints, fp)